# 搭建模型，训练并预测

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [60]:
# 加载数据
import pandas as pd
import os

from fastai.tabular import *
df = pd.read_csv("train_set.csv")
df = df.rename(columns={'告警开始时间_timestamp':'notice_time', '告警标题':'title', '故障原因定位（大类）':'label', '涉及基站eNBID或小区ECGI':'netid',
                  '故障发生时间_timestamp':'fault_time'})
df.drop(['告警开始时间'],inplace=True,axis=1)
df.drop(['fault_time'],inplace=True,axis=1)
df.drop(['notice_time'],inplace=True,axis=1)
df

,title,label,netid
0,小区不可用告警,电力故障,505346F7A6D536667EE8F73CD15C1179
1,小区接收通道干扰噪声功率不平衡告警,电力故障,597ACECFDC702C079940F608FA17BE76
2,小区接收通道干扰噪声功率不平衡告警,电力故障,F98EC4E70CBBD66A4005CC3EF9D8233D
3,4G基站退服告警,电力故障,8B52CD71E5397D8D818F641A56657863
4,时钟参考源异常告警,硬件故障,D28C3879A688B51BC7016E55131A3F2D
5,小区不可用告警,硬件故障,67E65EC1EE29F3971993424FD9DCEAA1
6,射频单元驻波告警,传输故障,476A1EBA3437F8117AD6C5D40C551FF7
7,射频单元CPRI接口异常告警,电力故障,B731EF07B1376AEE94C525C0ED8AEE88
8,网元连接中断,硬件故障,0C965544227EF071A4D6C247F77D0617
9,网元连接中断,电力故障,DB8AFB92757886A789D95AFB6D34300D


In [61]:
# df.groupby(["notice_time", "netid"])["title"].value_counts()

In [62]:
dep_var = 'label' # y
cat_names = ['title', 'netid']
cont_names = []
procs = [FillMissing, Categorify, Normalize]

In [63]:
path = Path('.')
test = TabularList.from_df(df.iloc[23690:27690].copy(), path=path, cat_names=cat_names, cont_names=cont_names)
data = (TabularList.from_df(df, path=path, cat_names=cat_names, cont_names=cont_names, procs=procs)
                           .split_by_idx(list(range(23690,27690)))
                           .label_from_df(cols=dep_var)
                           .add_test(test)
                           .databunch())

In [64]:
data.show_batch(rows=10)

title,netid,target
射频单元维护链路异常告警,6E34DD0A92DC6F0E6B902E2ADF94D7A7,电力故障
射频单元IR接口异常告警,6B79435B28F36874DC04EFA0DD4CB32B,硬件故障
同一机房多个基站退服告警,8AD736B1E78EB05DC128989E2CA7A2FB,电力故障
网元连接中断,EBCB5CB6A176488AFBFF25DE4D9F49EF,电力故障
射频单元业务不可用告警,67E29EB1F86E096A76C912171B536556,电力故障
射频单元交流掉电告警,97FCF8C52F0A2F8A6C1981ED5EB5F53D,电力故障
射频单元IR接口异常告警,C4C3096730E279CB2ACBFC8546285249,传输故障
射频单元硬件故障告警,0B4A42B96A3232C0AA0A88B7D4DB663A,电力故障
射频单元业务不可用告警,2603BF3EC7878D70C5A11CA50649910B,电力故障
BBU IR接口异常告警,96AD0DA5BB55EFF9BA2A345F3EF10BBB,传输故障


In [65]:
# def _acc(input, target):
#     target = target.squeeze(1)
#     mask = target != void_code
#     return (input.argmax(dim=1)[mask]==target[mask]).float().mean()

learn = tabular_learner(data, layers=[200,100],emb_szs={'native-country': 10}, metrics=[accuracy])

In [66]:
learn.fit(2, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,0.412589,0.611918,0.800250,00:08
1,0.303604,0.677027,0.800250,00:07


In [44]:
row = df.iloc[0]
row

notice_time                          1549296720
title                                   小区不可用告警
label                                      电力故障
netid          505346F7A6D536667EE8F73CD15C1179
Name: 0, dtype: object

In [52]:
learn.predict(df.iloc[101])

(Category 硬件故障,
 tensor(3),
 tensor([5.1708e-02, 7.3434e-04, 2.3343e-02, 9.2249e-01, 1.2381e-03, 4.8815e-04]))